In [1]:
import subprocess
import sys
import os
import re
import atexit
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import datetime as dt
from sklearn.preprocessing import OneHotEncoder

# Librería para evitar mensajes de warning al ejecutar código Python
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore')

# Librería para cronometrar tiempos de ejecución
#import time
#start_time = time.time()

# Definición de constantes
FIRST_DATE = dt.datetime(2012,12,1)
LAST_DATE = dt.datetime(2015,12,31)

## Carga de datos

In [2]:
# Diccionario con features y sus tipos
dtype = {'DeviceID':str, 'Fault_Code_Type_1':str, 'Fault_Code_Type_2':str, 'Fault_Code_Type_3':str, 'Fault_Code_Type_4':str}

# Carga del dataset teniendo en cuenta el diccionario definido y el formato de la feature de tipo fecha
df = pd.read_csv('sampledata.csv', sep=',', dtype=dtype, parse_dates=['Date'], infer_datetime_format=True)

## Preprocesamiento de datos

In [3]:
# Verificación del dataframe: dimesiones y tipo de sus features
print('Dimensiones del dataframe: {}'.format(df.shape))
print('\nTipo de las features:\n{}'.format(df.dtypes))

Dimensiones del dataframe: (2085102, 172)

Tipo de las features:
DeviceID                     object
Date                 datetime64[ns]
Categorical_1                 int64
Categorical_2                 int64
Categorical_3                 int64
Categorical_4                 int64
Problem_Type_1                int64
Usage_Count_1                 int64
Usage_Count_2                 int64
Problem_Type_2                int64
Problem_Type_3                int64
Problem_Type_4                int64
Warning_1.2                   int64
Warning_1.3                   int64
Warning_1.4                   int64
Warning_1.5                   int64
Warning_1.6                   int64
Warning_1.7                   int64
Warning_1.8                   int64
Warning_1.9                   int64
Warning_1.10                  int64
Warning_1.11                  int64
Warning_1.12                  int64
Warning_1.13                  int64
Warning_1.14                  int64
Warning_1.15                  int64

#### Definición de las features:
* ***DeviceID***: identificador de la máquina.
* ***Date***: fecha del registro (fila).
* ***Categorical_X***: variables categóricas de la máquina (x4).
* ***Problem_Type_X***: número total de problemas de un tipo reportados por la máquina durante un día (x4).
* ***Usage_Count_X***: número total de veces que la máquina ha sido usada en cada modo de funcionamiento (x2).
* ***Warning_X***: número total de avisos reportados por la máquina durante un día (x147).
* ***Error_Count_X***: número total de errores de un tipo reportados por la máquina durante un día (x8).
* ***Fault_Code_Type_X***: tipo de código de fallo reportado por la máquina durante un día (x4).
* ***ProblemReported***: feature objetivo (a predecir), que indica si la máquina tuvo un problema durante un día.

In [4]:
# Borrado de filas duplicadas cuyos valores coincidan en 'DeviceID' y 'Date'
df = df.drop_duplicates(['DeviceID', 'Date'])

# Borrado de filas en las que no exista valor en 'DeviceID' o 'Date'
df = df.dropna(how='any', subset=['DeviceID', 'Date'])

# Instrucción para hacer que la dupla 'DeviceID' y 'Date' sean los índices del dataframe
#df = df.set_index(['DeviceID', 'Date'])

# Creación de las features 'Day', 'Month' y 'Year', extraidas de la feature 'Date'
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [5]:
# Definición de grupos de features: date, categorical, numeric
datetime_features = ['Date', 'Day', 'Month', 'Year']
categorical_features = ['DeviceID', 'Categorical_1', 'Categorical_2', 'Categorical_3', 'Categorical_4',
                        'Fault_Code_Type_1', 'Fault_Code_Type_2', 'Fault_Code_Type_3', 'Fault_Code_Type_4']
warning_type1_features = [feature for feature in df.columns if feature.startswith('Warning_1')]
warning_type2_features = [feature for feature in df.columns if feature.startswith('Warning_2')]
warning_features = warning_type1_features + warning_type2_features
numeric_features = list(set(df.columns) - set(datetime_features) - set(categorical_features))

# Tratamiento de datos no registrados: valor 0 para features numéricas
values = dict(zip(numeric_features, np.zeros(len(numeric_features))))
df = df.fillna(value=values)

# Tratamiento de datos no registrados: valor 'Unknown' para features categóricas
values = dict(zip(categorical_features, ['Unknown' for k in range(len(categorical_features))]))
df = df.fillna(value=values)

# Visualización de los 5 primeros registros del dataframe
df.head()

DeviceID       Date  Categorical_1  Categorical_2  Categorical_3  \
0    N0001 2012-12-08              8           1500            100   
1    N0001 2012-12-09              8           1500            100   
2    N0001 2012-12-10              8           1500            100   
3    N0001 2012-12-11              8           1500            100   
4    N0001 2012-12-12              8           1500            100   

   Categorical_4  Problem_Type_1  Usage_Count_1  Usage_Count_2  \
0           7106               0              0              0   
1           7106               0              0              0   
2           7106               0              0              0   
3           7106               0              0              0   
4           7106               0              0              0   

   Problem_Type_2  ...   Error_Count_7  Error_Count_8  Fault_Code_Type_1  \
0               0  ...               0              0            Unknown   
1               0  ...               0              0            Unknown   
2               0  ...               0              0            Unknown   
3               0  ...               0              0            Unknown   
4               0  ...               0              0            Unknown   

   Fault_Code_Type_2  Fault_Code_Type_3  Fault_Code_Type_4  ProblemReported  \
0            Unknown            Unknown            Unknown                0   
1            Unknown            Unknown            Unknown                0   
2            Unknown            Unknown            Unknown                0   
3            Unknown            Unknown            Unknown                0   
4            Unknown            Unknown            Unknown                0   

   Day  Month  Year  
0    8     12  2012  
1    9     12  2012  
2   10     12  2012  
3   11     12  2012  
4   12     12  2012  

[5 rows x 175 columns]

In [6]:
# Detección de valores negativos en features que hacen de contadores
df[numeric_features].describe()

Warning_1.33  Warning_2.63  Warning_2.81  Warning_2.56  Warning_2.3  \
count     2085102.0  2.085102e+06  2.085102e+06     2085102.0    2085102.0   
mean          -10.0 -9.849034e+00 -9.994534e+00         -10.0        -10.0   
std             0.0  1.544099e+00  4.084725e-01           0.0          0.0   
min           -10.0 -1.000000e+01 -1.000000e+01         -10.0        -10.0   
25%           -10.0 -1.000000e+01 -1.000000e+01         -10.0        -10.0   
50%           -10.0 -1.000000e+01 -1.000000e+01         -10.0        -10.0   
75%           -10.0 -1.000000e+01 -1.000000e+01         -10.0        -10.0   
max           -10.0  4.790000e+02  2.580000e+02         -10.0        -10.0   

       Warning_2.93  Warning_2.57  Warning_2.64  Warning_2.29  Warning_1.12  \
count  2.085102e+06  2.085102e+06  2.085102e+06     2085102.0  2.085102e+06   
mean  -9.719726e+00 -9.999995e+00 -9.998427e+00         -10.0 -9.990561e+00   
std    2.302713e+00  7.617791e-03  1.345550e-01           0.0  3.359755e-01   
min   -1.000000e+01 -1.000000e+01 -1.000000e+01         -10.0 -1.000000e+01   
25%   -1.000000e+01 -1.000000e+01 -1.000000e+01         -10.0 -1.000000e+01   
50%   -1.000000e+01 -1.000000e+01 -1.000000e+01         -10.0 -1.000000e+01   
75%   -1.000000e+01 -1.000000e+01 -1.000000e+01         -10.0 -1.000000e+01   
max    4.050000e+02  1.000000e+00  7.000000e+00         -10.0  1.800000e+01   

           ...       Warning_2.49  ProblemReported  Warning_2.30  \
count      ...       2.085102e+06     2.085102e+06  2.085102e+06   
mean       ...      -9.967883e+00     1.910698e-03 -9.999953e+00   
std        ...       1.063396e+00     4.366976e-02  2.285333e-02   
min        ...      -1.000000e+01     0.000000e+00 -1.000000e+01   
25%        ...      -1.000000e+01     0.000000e+00 -1.000000e+01   
50%        ...      -1.000000e+01     0.000000e+00 -1.000000e+01   
75%        ...      -1.000000e+01     0.000000e+00 -1.000000e+01   
max        ...       2.800000e+02     1.000000e+00  1.000000e+00   

       Warning_2.52  Warning_2.20  Warning_2.67  Warning_2.78  Error_Count_4  \
count  2.085102e+06     2085102.0  2.085102e+06  2.085102e+06   2.085102e+06   
mean  -9.926698e+00         -10.0 -9.774329e+00 -9.949070e+00   4.316336e-04   
std    2.507259e+00           0.0  1.661925e+00  8.892755e-01   2.077131e-02   
min   -1.000000e+01         -10.0 -1.000000e+01 -1.000000e+01   0.000000e+00   
25%   -1.000000e+01         -10.0 -1.000000e+01 -1.000000e+01   0.000000e+00   
50%   -1.000000e+01         -10.0 -1.000000e+01 -1.000000e+01   0.000000e+00   
75%   -1.000000e+01         -10.0 -1.000000e+01 -1.000000e+01   0.000000e+00   
max    2.760000e+02         -10.0  2.570000e+02  2.760000e+02   1.000000e+00   

       Usage_Count_1  Warning_2.66  
count   2.085102e+06  2.085102e+06  
mean    3.816079e+02 -9.994782e+00  
std     5.104107e+02  2.481925e-01  
min    -3.266100e+04 -1.000000e+01  
25%     0.000000e+00 -1.000000e+01  
50%     6.800000e+01 -1.000000e+01  
75%     6.980000e+02 -1.000000e+01  
max     3.256500e+04  2.800000e+01  

[8 rows x 162 columns]

In [7]:
# Convertir valores negativos de todas las features numéricas (que hacen de contadores) en cero
for feature in numeric_features:
    df.loc[(df[feature] < 0), feature] = 0

# Comprobación
df[numeric_features].describe()

Warning_1.33  Warning_2.63  Warning_2.81  Warning_2.56  Warning_2.3  \
count     2085102.0  2.085102e+06  2.085102e+06     2085102.0    2085102.0   
mean            0.0  2.959088e-02  1.384585e-03           0.0          0.0   
std             0.0  7.749748e-01  3.136442e-01           0.0          0.0   
min             0.0  0.000000e+00  0.000000e+00           0.0          0.0   
25%             0.0  0.000000e+00  0.000000e+00           0.0          0.0   
50%             0.0  0.000000e+00  0.000000e+00           0.0          0.0   
75%             0.0  0.000000e+00  0.000000e+00           0.0          0.0   
max             0.0  4.790000e+02  2.580000e+02           0.0          0.0   

       Warning_2.93  Warning_2.57  Warning_2.64  Warning_2.29  Warning_1.12  \
count  2.085102e+06  2.085102e+06  2.085102e+06     2085102.0  2.085102e+06   
mean   4.945993e-02  4.795928e-07  1.966331e-04           0.0  1.362523e-03   
std    1.442656e+00  6.925264e-04  2.026063e-02           0.0  7.037627e-02   
min    0.000000e+00  0.000000e+00  0.000000e+00           0.0  0.000000e+00   
25%    0.000000e+00  0.000000e+00  0.000000e+00           0.0  0.000000e+00   
50%    0.000000e+00  0.000000e+00  0.000000e+00           0.0  0.000000e+00   
75%    0.000000e+00  0.000000e+00  0.000000e+00           0.0  0.000000e+00   
max    4.050000e+02  1.000000e+00  7.000000e+00           0.0  1.800000e+01   

           ...       Warning_2.49  ProblemReported  Warning_2.30  \
count      ...       2.085102e+06     2.085102e+06  2.085102e+06   
mean       ...       1.617043e-02     1.910698e-03  4.316336e-06   
std        ...       8.054235e-01     4.366976e-02  2.077575e-03   
min        ...       0.000000e+00     0.000000e+00  0.000000e+00   
25%        ...       0.000000e+00     0.000000e+00  0.000000e+00   
50%        ...       0.000000e+00     0.000000e+00  0.000000e+00   
75%        ...       0.000000e+00     0.000000e+00  0.000000e+00   
max        ...       2.800000e+02     1.000000e+00  1.000000e+00   

       Warning_2.52  Warning_2.20  Warning_2.67  Warning_2.78  Error_Count_4  \
count  2.085102e+06     2085102.0  2.085102e+06  2.085102e+06   2.085102e+06   
mean   4.252694e-02           0.0  3.515799e-02  9.685378e-03   4.316336e-04   
std    2.265307e+00           0.0  4.509899e-01  4.326121e-01   2.077131e-02   
min    0.000000e+00           0.0  0.000000e+00  0.000000e+00   0.000000e+00   
25%    0.000000e+00           0.0  0.000000e+00  0.000000e+00   0.000000e+00   
50%    0.000000e+00           0.0  0.000000e+00  0.000000e+00   0.000000e+00   
75%    0.000000e+00           0.0  0.000000e+00  0.000000e+00   0.000000e+00   
max    2.760000e+02           0.0  2.570000e+02  2.760000e+02   1.000000e+00   

       Usage_Count_1  Warning_2.66  
count   2.085102e+06  2.085102e+06  
mean    3.816666e+02  7.050015e-04  
std     5.093598e+02  4.895415e-02  
min     0.000000e+00  0.000000e+00  
25%     0.000000e+00  0.000000e+00  
50%     6.800000e+01  0.000000e+00  
75%     6.980000e+02  0.000000e+00  
max     3.256500e+04  2.800000e+01  

[8 rows x 162 columns]

## EDA

#### Selección de máquina, feature e intervalo de tiempo para graficar la tendencia:

In [8]:
def ExploratoryDataAnalysis(df, device, feature, start_date, end_date):
    if(device != None and feature != None):
        eda = df.loc[(df['DeviceID'] == device)]
        
        from_date = start_date if start_date != None else FIRST_DATE
        to_date = end_date if end_date != None else LAST_DATE
        
        if(pd.date_range(from_date, to_date).size > 0):
            eda = eda[eda['Date'].isin(pd.date_range(from_date, to_date))]
            
            eda.plot(x='Date', y=feature)

def f(device, feature, start_date, end_date):
    ExploratoryDataAnalysis(df, device, feature, start_date, end_date)

deviceID_selector = widgets.Dropdown(options=df.DeviceID.unique(), value=None, description='Device ID:', disabled=False)
feature_selector = widgets.Dropdown(options=numeric_features, value='Usage_Count_1', description='Feature:', disabled=False)
start_date_selector = widgets.DatePicker(value=FIRST_DATE, description='From:', disabled=False)
end_date_selector = widgets.DatePicker(value=LAST_DATE, description='To:', disabled=False)

w = widgets.interactive(f, device=deviceID_selector, feature=feature_selector,
                        start_date=start_date_selector, end_date=end_date_selector)

left_box = widgets.VBox([w.children[0], w.children[1]])
right_box = widgets.VBox([w.children[2], w.children[3]])
controls = widgets.HBox([left_box, right_box])
output = w.children[-1]
display(widgets.VBox([controls, output]))

#### Selección de máquina, dos features e intervalo de tiempo para graficar un scatter plot:

In [9]:
def ExploratoryDataAnalysis2(df, device, feature_x, feature_y, start_date, end_date):
    if(device != None and feature_x != None and feature_y != None):
        eda = df.loc[(df['DeviceID'] == device)]
        
        from_date = start_date if start_date != None else FIRST_DATE
        to_date = end_date if end_date != None else LAST_DATE
        
        if(pd.date_range(from_date, to_date).size > 0):
            eda = eda[eda['Date'].isin(pd.date_range(from_date, to_date))]
            
            eda.plot.scatter(x=feature_x, y=feature_y)

def f2(device, feature_x, feature_y, start_date, end_date):
    ExploratoryDataAnalysis2(df, device, feature_x, feature_y, start_date, end_date)

deviceID_selector = widgets.Dropdown(options=df.DeviceID.unique(), value=None, description='Device ID:', disabled=False)
feature_x_selector = widgets.Dropdown(options=numeric_features, value='Usage_Count_1', description='X-Feature:', disabled=False)
feature_y_selector = widgets.Dropdown(options=numeric_features, value='Usage_Count_2', description='Y-Feature:', disabled=False)
start_date_selector = widgets.DatePicker(value=FIRST_DATE, description='From:', disabled=False)
end_date_selector = widgets.DatePicker(value=LAST_DATE, description='To:', disabled=False)

w = widgets.interactive(f2, device=deviceID_selector, feature_x=feature_x_selector, feature_y=feature_y_selector,
                        start_date=start_date_selector, end_date=end_date_selector)

left_box = widgets.VBox([w.children[0]])
center_box = widgets.VBox([w.children[1], w.children[2]])
right_box = widgets.VBox([w.children[3], w.children[4]])
controls = widgets.HBox([left_box, center_box, right_box])
output = w.children[-1]
display(widgets.VBox([controls, output]))

#### Heatmap con el acumulado de una feature seleccionada por cada mes y año:

In [10]:
def ExploratoryDataAnalysis3(df, feature):            
    eda = pd.pivot_table(df, values=feature, index='Year', columns='Month', aggfunc=np.sum)
    plt.figure(figsize = (16,4))
    plt.title('Feature acumulada por mes y año', fontsize=20)
    sns.heatmap(eda, cmap="YlGnBu", annot=True, fmt='.0f')

def f3(feature):
    ExploratoryDataAnalysis3(df, feature)

feature_selector = widgets.Dropdown(options=numeric_features, value='ProblemReported', description='Feature:', disabled=False)

w = widgets.interactive(f3, feature=feature_selector)
display(w)

interactive(children=(Dropdown(description='Feature:', index=153, options=('Warning_1.33', 'Warning_2.63', 'Wa…

In [11]:
# Visualización de un heatmap con la matriz de correlación entre features numéricas (sin warnings)
df_corr = df[['Error_Count_1', 'Error_Count_2', 'Error_Count_3', 'Error_Count_4',
              'Error_Count_5', 'Error_Count_6', 'Error_Count_7', 'Error_Count_8', 'ProblemReported',
            'Problem_Type_1', 'Problem_Type_2', 'Problem_Type_3', 'Problem_Type_4', 'Usage_Count_1', 'Usage_Count_2']]
corr = df_corr.corr()
plt.figure(figsize = (16,4))
plt.title('Matriz de correlación entre features numéricas', fontsize=20)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, cmap="YlGnBu", annot=True, fmt='.2f')

## Feature Engineering

#### Warnings

In [12]:
# Creación de dos features: suma de las features de tipo Warning_1 y suma de as features de tipo Warning_2
df['Warning_Type_1'] = sum(df[feature] for feature in warning_type1_features)
df['Warning_Type_2'] = sum(df[feature] for feature in warning_type2_features)

#### Categorical_X

In [13]:
# Visualización de un heatmap con los valores de las features 'Categorical_X' por cada 'DeviceID'
pt1 = pd.pivot_table(df, values=['Categorical_1', 'Categorical_2', 'Categorical_3', 'Categorical_4'],
                     columns='DeviceID', aggfunc=np.std)
plt.figure(figsize = (16,4))
plt.title('Las 4 features \'Categorical\' mantienen un valor constante en el tiempo por cada máquina')
sns.heatmap(pt1)

In [14]:
df_cat = df[['Categorical_1', 'Categorical_2', 'Categorical_3', 'Categorical_4']].describe()
df_cat.round(1)

Categorical_1  Categorical_2  Categorical_3  Categorical_4
count      2085102.0      2085102.0      2085102.0      2085102.0
mean             9.1         3403.9          315.2         9630.1
std              4.9          878.9          111.3         5500.0
min              2.0          800.0           50.0          600.0
25%              5.0         2500.0          200.0         5600.0
50%              8.0         3500.0          350.0         8404.0
75%             12.0         4000.0          350.0        12400.0
max             37.0        10000.0          800.0        37406.0

In [15]:
# Creación de 4 features categóricas a partir de las 'Categorical_X', definiendo entre 3 y 4 categorías en cada una de ellas
df['Categorical_1_Grouped'] = pd.qcut(df['Categorical_1'], 4, labels=['LL', 'L', 'H', 'HH'])
df['Categorical_2_Grouped'] = pd.qcut(df['Categorical_2'], 4, labels=['LL', 'L', 'H', 'HH'])
df['Categorical_3_Grouped'] = pd.qcut(df['Categorical_3'], 3, labels=['LL', 'L', 'H'])
df['Categorical_4_Grouped'] = pd.qcut(df['Categorical_4'], 4, labels=['LL', 'L', 'H', 'HH'])

df_cat = df[['Categorical_1_Grouped', 'Categorical_2_Grouped', 'Categorical_3_Grouped', 'Categorical_4_Grouped']].describe()
df_cat.round(1)

Categorical_1_Grouped Categorical_2_Grouped Categorical_3_Grouped  \
count                2085102               2085102               2085102   
unique                     4                     4                     3   
top                        L                     L                     L   
freq                  631350                972828               1152900   

       Categorical_4_Grouped  
count                2085102  
unique                     4  
top                        H  
freq                  524844

#### Problem_Type_X

In [16]:
df_pbm = df[['Problem_Type_1', 'Problem_Type_2', 'Problem_Type_3', 'Problem_Type_4']].describe()
df_pbm.round(1)

Problem_Type_1  Problem_Type_2  Problem_Type_3  Problem_Type_4
count       2085102.0       2085102.0       2085102.0       2085102.0
mean              0.1            27.0             0.0             0.1
std               0.8            55.5             0.0             5.5
min               0.0             0.0             0.0             0.0
25%               0.0             0.0             0.0             0.0
50%               0.0             2.0             0.0             0.0
75%               0.0            34.0             0.0             0.0
max              80.0          5649.0            20.0           790.0

#### Error_Count_X

In [17]:
df_err = df[['Error_Count_1', 'Error_Count_2', 'Error_Count_3', 'Error_Count_4',
            'Error_Count_5', 'Error_Count_6', 'Error_Count_7', 'Error_Count_8']].describe()
df_err.round(1)

Error_Count_1  Error_Count_2  Error_Count_3  Error_Count_4  \
count      2085102.0      2085102.0      2085102.0      2085102.0   
mean             0.0            0.0            0.0            0.0   
std              0.1            0.0            0.0            0.0   
min              0.0            0.0            0.0            0.0   
25%              0.0            0.0            0.0            0.0   
50%              0.0            0.0            0.0            0.0   
75%              0.0            0.0            0.0            0.0   
max              1.0            1.0            1.0            1.0   

       Error_Count_5  Error_Count_6  Error_Count_7  Error_Count_8  
count      2085102.0      2085102.0      2085102.0      2085102.0  
mean             0.0            0.0            0.0            0.0  
std              0.1            0.0            0.0            0.0  
min              0.0            0.0            0.0            0.0  
25%              0.0            0.0            0.0            0.0  
50%              0.0            0.0            0.0            0.0  
75%              0.0            0.0            0.0            0.0  
max              2.0            1.0            1.0            1.0

In [18]:
df['Error_Count_5'] = [1 if x > 1 else x for x in df['Error_Count_5']]
df[['Error_Count_5']].describe().round(1)

Error_Count_5
count      2085102.0
mean             0.0
std              0.1
min              0.0
25%              0.0
50%              0.0
75%              0.0
max              1.0

#### Fault_Code_Type_X

In [19]:
df_flt = df[['Fault_Code_Type_1', 'Fault_Code_Type_2', 'Fault_Code_Type_3', 'Fault_Code_Type_4']].describe()
df_flt

Fault_Code_Type_1 Fault_Code_Type_2 Fault_Code_Type_3 Fault_Code_Type_4
count            2085102           2085102           2085102           2085102
unique                33                16                57                22
top              Unknown           Unknown           Unknown           Unknown
freq             2034466           2019426           1998693           2045509

In [20]:
# Transformación de las features 'Fault_Code_type_X' de texto a entero
df['Fault_Code_Type_1_Count'] = [0 if x == 'Unknown' else 1 for x in df['Fault_Code_Type_1']]
df['Fault_Code_Type_2_Count'] = [0 if x == 'Unknown' else 1 for x in df['Fault_Code_Type_2']]
df['Fault_Code_Type_3_Count'] = [0 if x == 'Unknown' else 1 for x in df['Fault_Code_Type_3']]
df['Fault_Code_Type_4_Count'] = [0 if x == 'Unknown' else 1 for x in df['Fault_Code_Type_4']]

#### Creación de ratios que relativicen la frecuencia de aparición de avisos, problemas, fallos y errores en función del uso dado a la máquina:

In [21]:
# Ratios 'Problem_Type_X' / 'Usage_Count_X'
df['Problem_Type_1_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Problem_Type_1'] / df['Usage_Count_1'])
df['Problem_Type_2_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Problem_Type_2'] / df['Usage_Count_1'])
df['Problem_Type_3_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Problem_Type_3'] / df['Usage_Count_1'])
df['Problem_Type_4_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Problem_Type_4'] / df['Usage_Count_1'])
df['Problem_Type_1_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Problem_Type_1'] / df['Usage_Count_2'])
df['Problem_Type_2_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Problem_Type_2'] / df['Usage_Count_2'])
df['Problem_Type_3_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Problem_Type_3'] / df['Usage_Count_2'])
df['Problem_Type_4_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Problem_Type_4'] / df['Usage_Count_2'])

# Ratios 'Fault_Code_Type_X_Count' / 'Usage_Count_X'
df['Fault_Code_Type_1_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Fault_Code_Type_1_Count'] / df['Usage_Count_1'])
df['Fault_Code_Type_2_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Fault_Code_Type_2_Count'] / df['Usage_Count_1'])
df['Fault_Code_Type_3_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Fault_Code_Type_3_Count'] / df['Usage_Count_1'])
df['Fault_Code_Type_4_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Fault_Code_Type_4_Count'] / df['Usage_Count_1'])
df['Fault_Code_Type_1_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Fault_Code_Type_1_Count'] / df['Usage_Count_2'])
df['Fault_Code_Type_2_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Fault_Code_Type_2_Count'] / df['Usage_Count_2'])
df['Fault_Code_Type_3_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Fault_Code_Type_3_Count'] / df['Usage_Count_2'])
df['Fault_Code_Type_4_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Fault_Code_Type_4_Count'] / df['Usage_Count_2'])

# Ratios 'Warning_Type_X' / 'Usage_Count_X'
df['Warning_Type_1_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Warning_Type_1'] / df['Usage_Count_1'])
df['Warning_Type_2_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df['Warning_Type_2'] / df['Usage_Count_1'])
df['Warning_Type_1_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Warning_Type_1'] / df['Usage_Count_2'])
df['Warning_Type_2_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df['Warning_Type_2'] / df['Usage_Count_2'])

#### Procesado de los ratios: discretización por categorías (_Grouped) y escalado entre [0, 1] con normalización min-max

In [22]:
# Transformación de las features recientemente creadas a categóricas
ratio_features = [feature for feature in df.columns if 'Per_Usage' in feature]

for feature in ratio_features:
    df[feature + '_Grouped'] = ['>1' if x > 1 else '>0' if x > 0 else 'Zero' for x in df[feature]]

# Creación de un dataframe con todas las features categóricas
features_grouped = [feature for feature in df.columns if '_Grouped' in feature]
features_grouped.extend(['Error_Count_1', 'Error_Count_2', 'Error_Count_3', 'Error_Count_4',
                         'Error_Count_5', 'Error_Count_6', 'Error_Count_7', 'Error_Count_8'])
df_grp = df[features_grouped]
df_grp.head()

Categorical_1_Grouped Categorical_2_Grouped Categorical_3_Grouped  \
0                     L                    LL                    LL   
1                     L                    LL                    LL   
2                     L                    LL                    LL   
3                     L                    LL                    LL   
4                     L                    LL                    LL   

  Categorical_4_Grouped Problem_Type_1_Per_Usage_1_Grouped  \
0                     L                               Zero   
1                     L                               Zero   
2                     L                               Zero   
3                     L                               Zero   
4                     L                               Zero   

  Problem_Type_2_Per_Usage_1_Grouped Problem_Type_3_Per_Usage_1_Grouped  \
0                               Zero                               Zero   
1                               Zero                               Zero   
2                               Zero                               Zero   
3                               Zero                               Zero   
4                               Zero                               Zero   

  Problem_Type_4_Per_Usage_1_Grouped Problem_Type_1_Per_Usage_2_Grouped  \
0                               Zero                               Zero   
1                               Zero                               Zero   
2                               Zero                               Zero   
3                               Zero                               Zero   
4                               Zero                               Zero   

  Problem_Type_2_Per_Usage_2_Grouped      ...       \
0                               Zero      ...        
1                               Zero      ...        
2                               Zero      ...        
3                               Zero      ...        
4                               Zero      ...        

  Warning_Type_1_Per_Usage_2_Grouped Warning_Type_2_Per_Usage_2_Grouped  \
0                               Zero                               Zero   
1                               Zero                               Zero   
2                               Zero                               Zero   
3                               Zero                               Zero   
4                               Zero                               Zero   

  Error_Count_1 Error_Count_2 Error_Count_3 Error_Count_4 Error_Count_5  \
0             0             0             0             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             0             0             0             0             0   
4             0             0             0             0             0   

  Error_Count_6 Error_Count_7 Error_Count_8  
0             0             0             0  
1             0             0             0  
2             0             0             0  
3             0             0             0  
4             0             0             0  

[5 rows x 32 columns]

In [23]:
# One-Hot Encoder
df_grp_ohe = pd.DataFrame()

for feature in df_grp.columns:
    temp = pd.get_dummies(df_grp[feature], prefix=feature)
    df_grp_ohe[temp.columns] = temp

df_grp_ohe.head()

Categorical_1_Grouped_LL  Categorical_1_Grouped_L  Categorical_1_Grouped_H  \
0                         0                        1                        0   
1                         0                        1                        0   
2                         0                        1                        0   
3                         0                        1                        0   
4                         0                        1                        0   

   Categorical_1_Grouped_HH  Categorical_2_Grouped_LL  \
0                         0                         1   
1                         0                         1   
2                         0                         1   
3                         0                         1   
4                         0                         1   

   Categorical_2_Grouped_L  Categorical_2_Grouped_H  Categorical_2_Grouped_HH  \
0                        0                        0                         0   
1                        0                        0                         0   
2                        0                        0                         0   
3                        0                        0                         0   
4                        0                        0                         0   

   Categorical_3_Grouped_LL  Categorical_3_Grouped_L       ...         \
0                         1                        0       ...          
1                         1                        0       ...          
2                         1                        0       ...          
3                         1                        0       ...          
4                         1                        0       ...          

   Error_Count_4_0  Error_Count_4_1  Error_Count_5_0  Error_Count_5_1  \
0                1                0                1                0   
1                1                0                1                0   
2                1                0                1                0   
3                1                0                1                0   
4                1                0                1                0   

   Error_Count_6_0  Error_Count_6_1  Error_Count_7_0  Error_Count_7_1  \
0                1                0                1                0   
1                1                0                1                0   
2                1                0                1                0   
3                1                0                1                0   
4                1                0                1                0   

   Error_Count_8_0  Error_Count_8_1  
0                1                0  
1                1                0  
2                1                0  
3                1                0  
4                1                0  

[5 rows x 83 columns]

In [24]:
# Eliminación de las features '_Grouped'
print('Número de features previo:', df.shape[1])
df = df.drop([feature for feature in df.columns if '_Grouped' in feature], axis=1)
print('Número de features tras la eliminación:', df.shape[1])

# Inclusión de las nuevas features OHE al dataset
df = pd.concat([df, df_grp_ohe], axis=1)
print('Número de features tras la inclusión:', df.shape[1])

Número de features previo: 225
Número de features tras la eliminación: 201
Número de features tras la inclusión: 284


In [25]:
# Escalado mediante normalización Min-Max
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

for feature in ratio_features:
    df[feature] = scaler.fit_transform(np.array(df[feature]).reshape(-1, 1))

df[[feature for feature in ratio_features]].describe()

Problem_Type_1_Per_Usage_1  Problem_Type_2_Per_Usage_1  \
count                2.085102e+06                2.085102e+06   
mean                 3.035300e-04                1.830884e-04   
std                  8.064972e-03                8.766719e-04   
min                  0.000000e+00                0.000000e+00   
25%                  0.000000e+00                0.000000e+00   
50%                  0.000000e+00                7.797879e-05   
75%                  0.000000e+00                2.780868e-04   
max                  1.000000e+00                1.000000e+00   

       Problem_Type_3_Per_Usage_1  Problem_Type_4_Per_Usage_1  \
count                2.085102e+06                2.085102e+06   
mean                 3.870221e-06                2.598609e-04   
std                  1.070702e-03                1.430753e-02   
min                  0.000000e+00                0.000000e+00   
25%                  0.000000e+00                0.000000e+00   
50%                  0.000000e+00                0.000000e+00   
75%                  0.000000e+00                0.000000e+00   
max                  1.000000e+00                1.000000e+00   

       Problem_Type_1_Per_Usage_2  Problem_Type_2_Per_Usage_2  \
count                2.085102e+06                2.085102e+06   
mean                 6.619626e-05                6.294360e-05   
std                  2.017468e-03                2.109286e-03   
min                  0.000000e+00                0.000000e+00   
25%                  0.000000e+00                0.000000e+00   
50%                  0.000000e+00                0.000000e+00   
75%                  0.000000e+00                4.715306e-05   
max                  1.000000e+00                1.000000e+00   

       Problem_Type_3_Per_Usage_2  Problem_Type_4_Per_Usage_2  \
count                2.085102e+06                2.085102e+06   
mean                 1.601723e-05                8.053779e-06   
std                  2.058868e-03                1.328000e-03   
min                  0.000000e+00                0.000000e+00   
25%                  0.000000e+00                0.000000e+00   
50%                  0.000000e+00                0.000000e+00   
75%                  0.000000e+00                0.000000e+00   
max                  1.000000e+00                1.000000e+00   

       Fault_Code_Type_1_Per_Usage_1  Fault_Code_Type_2_Per_Usage_1  \
count                   2.085102e+06                   2.085102e+06   
mean                    1.971076e-04                   1.858047e-04   
std                     8.440072e-03                   7.413326e-03   
min                     0.000000e+00                   0.000000e+00   
25%                     0.000000e+00                   0.000000e+00   
50%                     0.000000e+00                   0.000000e+00   
75%                     0.000000e+00                   0.000000e+00   
max                     1.000000e+00                   1.000000e+00   

       Fault_Code_Type_3_Per_Usage_1  Fault_Code_Type_4_Per_Usage_1  \
count                   2.085102e+06                   2.085102e+06   
mean                    2.131538e-04                   7.190571e-05   
std                     7.779539e-03                   4.327456e-03   
min                     0.000000e+00                   0.000000e+00   
25%                     0.000000e+00                   0.000000e+00   
50%                     0.000000e+00                   0.000000e+00   
75%                     0.000000e+00                   0.000000e+00   
max                     1.000000e+00                   1.000000e+00   

       Fault_Code_Type_1_Per_Usage_2  Fault_Code_Type_2_Per_Usage_2  \
count                   2.085102e+06                   2.085102e+06   
mean                    2.430517e-04                   3.331298e-04   
std                     9.722748e-03                   1.292298e-02   
min                     0.000000e+00                   0.000000e+00   
25%                     0.00

#### 4 AutoEncoders para reducir el número de features (en lugar de PCA)

In [26]:
# Creación de las features que pasarán por los 4 autoencoders: un autoencoder para cada grupo de warnings y modo de uso
df_ae_t1_u1 = pd.DataFrame()
df_ae_t1_u2 = pd.DataFrame()
df_ae_t2_u1 = pd.DataFrame()
df_ae_t2_u2 = pd.DataFrame()

for feature in warning_type1_features:
    df_ae_t1_u1[feature + '_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df[feature] / df['Usage_Count_1'])
    df_ae_t1_u2[feature + '_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df[feature] / df['Usage_Count_2'])

for feature in warning_type2_features:
    df_ae_t2_u1[feature + '_Per_Usage_1'] = np.where(df['Usage_Count_1'] == 0, 0, df[feature] / df['Usage_Count_1'])
    df_ae_t2_u2[feature + '_Per_Usage_2'] = np.where(df['Usage_Count_2'] == 0, 0, df[feature] / df['Usage_Count_2'])

In [27]:
# Normalización Min-Max
scaler = MinMaxScaler()
for column in df_ae_t1_u1.columns:
    df_ae_t1_u1[column] = scaler.fit_transform(np.array(df_ae_t1_u1[column]).reshape(-1, 1))

for column in df_ae_t1_u2.columns:
    df_ae_t1_u2[column] = scaler.fit_transform(np.array(df_ae_t1_u2[column]).reshape(-1, 1))

for column in df_ae_t2_u1.columns:
    df_ae_t2_u1[column] = scaler.fit_transform(np.array(df_ae_t2_u1[column]).reshape(-1, 1))

for column in df_ae_t2_u2.columns:
    df_ae_t2_u2[column] = scaler.fit_transform(np.array(df_ae_t2_u2[column]).reshape(-1, 1))

In [28]:
from keras.models import Model
from keras.layers import Input, Dense

# Definimos la semilla aleatoria para fijar los resultados obtenidos
seed = 2019
np.random.seed(seed)
AE_FEATURES_NUM = 5

# AutoEncoder 1: Modelado del autoencoder mediante la API funcional
entrada = Input(shape=(df_ae_t1_u1.shape[1],))
codificado = Dense(AE_FEATURES_NUM, activation='relu', name='capa_intermedia')(entrada)
decodificado = Dense(df_ae_t1_u1.shape[1], activation='relu')(codificado)
autoencoder = Model(entrada, decodificado)

# AutoEncoder 1: Modelado del encoder para obtener los datos compactos
encoder1 = Model(autoencoder.input, autoencoder.get_layer('capa_intermedia').output)

# AutoEncoder 1: Construcción y entrenamiento del autoencoder
autoencoder.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
autoencoder.fit(df_ae_t1_u1, df_ae_t1_u1, epochs=3)

C:\Users\ayfdcp\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
2085102/2085102 [==============================] - 50s 24us/step - loss: 5.7278e-06 - acc: 0.8291
Epoch 2/3
2085102/2085102 [==============================] - 48s 23us/step - loss: 5.6689e-06 - acc: 0.8527
Epoch 3/3
2085102/2085102 [==============================] - 46s 22us/step - loss: 5.6540e-06 - acc: 0.8586


In [29]:
# AutoEncoder 2: Modelado del autoencoder mediante la API funcional
entrada = Input(shape=(df_ae_t1_u2.shape[1],))
codificado = Dense(AE_FEATURES_NUM, activation='relu', name='capa_intermedia')(entrada)
decodificado = Dense(df_ae_t1_u2.shape[1], activation='relu')(codificado)
autoencoder = Model(entrada, decodificado)

# AutoEncoder 2: Modelado del encoder para obtener los datos compactos
encoder2 = Model(autoencoder.input, autoencoder.get_layer('capa_intermedia').output)

# AutoEncoder 2: Construcción y entrenamiento del autoencoder
autoencoder.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
autoencoder.fit(df_ae_t1_u2, df_ae_t1_u2, epochs=3)

Epoch 1/3
2085102/2085102 [==============================] - 47s 23us/step - loss: 1.0533e-06 - acc: 0.7999
Epoch 2/3
2085102/2085102 [==============================] - 51s 24us/step - loss: 9.6705e-07 - acc: 0.8034
Epoch 3/3
2085102/2085102 [==============================] - 57s 27us/step - loss: 9.5262e-07 - acc: 0.8069


In [30]:
# AutoEncoder 3: Modelado del autoencoder mediante la API funcional
entrada = Input(shape=(df_ae_t2_u1.shape[1],))
codificado = Dense(AE_FEATURES_NUM, activation='relu', name='capa_intermedia')(entrada)
decodificado = Dense(df_ae_t2_u1.shape[1], activation='relu')(codificado)
autoencoder = Model(entrada, decodificado)

# AutoEncoder 3: Modelado del encoder para obtener los datos compactos
encoder3 = Model(autoencoder.input, autoencoder.get_layer('capa_intermedia').output)

# AutoEncoder 3: Construcción y entrenamiento del autoencoder
autoencoder.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
autoencoder.fit(df_ae_t2_u1, df_ae_t2_u1, epochs=3)

Epoch 1/3
2085102/2085102 [==============================] - 76s 37us/step - loss: 1.0783e-06 - acc: 0.7921
Epoch 2/3
2085102/2085102 [==============================] - 68s 33us/step - loss: 1.0685e-06 - acc: 0.7966
Epoch 3/3
2085102/2085102 [==============================] - 67s 32us/step - loss: 1.0609e-06 - acc: 0.7956


In [31]:
# AutoEncoder 4: Modelado del autoencoder mediante la API funcional
entrada = Input(shape=(df_ae_t2_u2.shape[1],))
codificado = Dense(AE_FEATURES_NUM, activation='relu', name='capa_intermedia')(entrada)
decodificado = Dense(df_ae_t2_u2.shape[1], activation='relu')(codificado)
autoencoder = Model(entrada, decodificado)

# AutoEncoder 4: Modelado del encoder para obtener los datos compactos
encoder4 = Model(autoencoder.input, autoencoder.get_layer('capa_intermedia').output)

# AutoEncoder 4: Construcción y entrenamiento del autoencoder
autoencoder.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
autoencoder.fit(df_ae_t2_u2, df_ae_t2_u2, epochs=3)

Epoch 1/3
2085102/2085102 [==============================] - 67s 32us/step - loss: 1.2246e-06 - acc: 0.8061
Epoch 2/3
2085102/2085102 [==============================] - 68s 33us/step - loss: 1.2002e-06 - acc: 0.8106
Epoch 3/3
2085102/2085102 [==============================] - 71s 34us/step - loss: 1.1582e-06 - acc: 0.8089


In [32]:
# Eliminación de las features 'Warning_X.X'
df = df.drop([feature for feature in warning_features], axis=1)
print('Número de features tras la eliminación:', df.shape[1])

# Obtención de las 20 features comprimidas por el encoder
df_ae_t1_u1 = encoder1.predict(df_ae_t1_u1)
df_ae_t1_u1 = pd.DataFrame(df_ae_t1_u1, columns=['AE_T1U1_Warning_' + str(i + 1) for i in range(AE_FEATURES_NUM)])

df_ae_t1_u2 = encoder2.predict(df_ae_t1_u2)
df_ae_t1_u2 = pd.DataFrame(df_ae_t1_u2, columns=['AE_T1U2_Warning_' + str(i + 1) for i in range(AE_FEATURES_NUM)])

df_ae_t2_u1 = encoder3.predict(df_ae_t2_u1)
df_ae_t2_u1 = pd.DataFrame(df_ae_t2_u1, columns=['AE_T2U1_Warning_' + str(i + 1) for i in range(AE_FEATURES_NUM)])

df_ae_t2_u2 = encoder4.predict(df_ae_t2_u2)
df_ae_t2_u2 = pd.DataFrame(df_ae_t2_u2, columns=['AE_T2U2_Warning_' + str(i + 1) for i in range(AE_FEATURES_NUM)])

# Normalización Min-Max de los resultados compactos
scaler = MinMaxScaler()
for column in df_ae_t1_u1.columns:
    df_ae_t1_u1[column] = scaler.fit_transform(np.array(df_ae_t1_u1[column]).reshape(-1, 1))

for column in df_ae_t1_u2.columns:
    df_ae_t1_u2[column] = scaler.fit_transform(np.array(df_ae_t1_u2[column]).reshape(-1, 1))

for column in df_ae_t2_u1.columns:
    df_ae_t2_u1[column] = scaler.fit_transform(np.array(df_ae_t2_u1[column]).reshape(-1, 1))

for column in df_ae_t2_u2.columns:
    df_ae_t2_u2[column] = scaler.fit_transform(np.array(df_ae_t2_u2[column]).reshape(-1, 1))

# Inclusión de las nuevas features al dataset
df = pd.concat([df, df_ae_t1_u1, df_ae_t1_u2, df_ae_t2_u1, df_ae_t2_u2], axis=1)
print('Número de features tras la inclusión:', df.shape[1])

Número de features tras la eliminación: 137
Número de features tras la inclusión: 157


## Guardado de datos

In [33]:
df.to_pickle('./sampledata2.pkl')